In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14254,2022-01-02T17:00:00,ITA,19,Sicilia,38.115697,13.362357,811,107,918,50378,...,DECEDUTI: N. 4 IL 01/01/22 - N. 9 IL 31/12/21,9.0,NaN,NaN,382332.0,0.0,3857408.0,4628473.0,ITG,ITG1
14255,2022-01-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,695,85,780,94017,...,NaN,9.0,NaN,NaN,370906.0,32054.0,5489260.0,3737823.0,ITI,ITI1
14256,2022-01-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,151,8,159,23204,...,NaN,1.0,NaN,NaN,84524.0,8944.0,1355436.0,1570704.0,ITI,ITI2
14257,2022-01-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,36,3,39,2554,...,NaN,0.0,NaN,NaN,13624.0,3067.0,126217.0,235667.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14254,2022-01-02T17:00:00,19,Sicilia,811,107,918,50378,51296,3531,3964,...,382332,8485881,3050221.0,9.0,382332.0,0.0,3857408.0,4628473.0,ITG,ITG1
14255,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,402960,9227083,3689948.0,9.0,370906.0,32054.0,5489260.0,3737823.0,ITI,ITI1
14256,2022-01-02T17:00:00,10,Umbria,151,8,159,23204,23363,539,788,...,93468,2926140,538529.0,1.0,84524.0,8944.0,1355436.0,1570704.0,ITI,ITI2
14257,2022-01-02T17:00:00,2,Valle d'Aosta,36,3,39,2554,2593,114,117,...,16691,361884,111580.0,0.0,13624.0,3067.0,126217.0,235667.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-02', '2022-01-01')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-02T17:00:00,1,Piemonte,1294,109,1403,78688,80091,1551,2619,...,504838,11421200,3100684.0,6.0,403025.0,101813.0,4172478.0,7248722.0,ITC,ITC1
1,2022-01-02T17:00:00,2,Valle d'Aosta,36,3,39,2554,2593,114,117,...,16691,361884,111580.0,0.0,13624.0,3067.0,126217.0,235667.0,ITC,ITC2
2,2022-01-02T17:00:00,3,Lombardia,2052,216,2268,289825,292093,7196,10425,...,1265059,25183247,6868839.0,15.0,993570.0,271489.0,13094013.0,12089234.0,ITC,ITC4
3,2022-01-02T17:00:00,5,Veneto,1120,188,1308,105491,106799,2908,3816,...,663809,19758438,2472640.0,16.0,592987.0,70822.0,8333719.0,11424719.0,ITH,ITH3
4,2022-01-02T17:00:00,6,Friuli Venezia Giulia,298,28,326,14440,14766,918,1188,...,159605,4556503,969305.0,4.0,138151.0,21454.0,2661326.0,1895177.0,ITH,ITH4
5,2022-01-02T17:00:00,7,Liguria,510,47,557,13302,13859,427,1265,...,152757,3573911,1052169.0,3.0,152757.0,0.0,2006357.0,1567554.0,ITC,ITC3
6,2022-01-02T17:00:00,8,Emilia-Romagna,1451,130,1581,95471,97052,8186,9090,...,558263,11738029,2407107.0,10.0,547665.0,10598.0,7329428.0,4408601.0,ITH,ITH5
7,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,402960,9227083,3689948.0,9.0,370906.0,32054.0,5489260.0,3737823.0,ITI,ITI1
8,2022-01-02T17:00:00,10,Umbria,151,8,159,23204,23363,539,788,...,93468,2926140,538529.0,1.0,84524.0,8944.0,1355436.0,1570704.0,ITI,ITI2
9,2022-01-02T17:00:00,11,Marche,227,50,277,7832,8109,348,699,...,147782,2030264,1194317.0,4.0,147782.0,0.0,1654957.0,375307.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-01T17:00:00,1,Piemonte,1242,108,1350,77190,78540,5722,7450,...,502219,11404748,3096619.0,12.0,401949.0,100270.0,4166430.0,7238318.0,ITC,ITC1
1,2022-01-01T17:00:00,2,Valle d'Aosta,32,3,35,2444,2479,350,383,...,16574,360737,111003.0,0.0,13626.0,2948.0,126191.0,234546.0,ITC,ITC2
2,2022-01-01T17:00:00,3,Lombardia,1945,222,2167,282730,284897,32517,37270,...,1254634,25139075,6861289.0,25.0,985733.0,268901.0,13064951.0,12074124.0,ITC,ITC4
3,2022-01-01T17:00:00,5,Veneto,1134,184,1318,102573,103891,10593,14270,...,659993,19733474,2469045.0,20.0,590400.0,69593.0,8321068.0,11412406.0,ITH,ITH3
4,2022-01-01T17:00:00,6,Friuli Venezia Giulia,275,28,303,13545,13848,1328,2331,...,158419,4551164,968465.0,5.0,137009.0,21410.0,2657044.0,1894120.0,ITH,ITH4
5,2022-01-01T17:00:00,7,Liguria,494,48,542,12890,13432,818,2432,...,151492,3567682,1050936.0,6.0,151492.0,0.0,2002034.0,1565648.0,ITC,ITC3
6,2022-01-01T17:00:00,8,Emilia-Romagna,1359,124,1483,87383,88866,11408,12255,...,549175,11720539,2401783.0,16.0,541890.0,7285.0,7316502.0,4404037.0,ITH,ITH5
7,2022-01-01T17:00:00,9,Toscana,653,81,734,88142,88876,14576,14994,...,396593,9204570,3677834.0,10.0,365561.0,31032.0,5471303.0,3733267.0,ITI,ITI1
8,2022-01-01T17:00:00,10,Umbria,133,8,141,22683,22824,3421,3838,...,92680,2923395,537764.0,0.0,84059.0,8621.0,1354936.0,1568459.0,ITI,ITI2
9,2022-01-01T17:00:00,11,Marche,223,47,270,7491,7761,121,1619,...,147083,2026805,1192226.0,5.0,147083.0,0.0,1652434.0,374371.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-02T17:00:00,1,Piemonte,1294,109,1403,78688,80091,1551,2619,...,11421200,3100684.0,6.0,403025.0,101813.0,4172478.0,7248722.0,ITC,ITC1,4.341375e+06
1,2022-01-02T17:00:00,2,Valle d'Aosta,36,3,39,2554,2593,114,117,...,361884,111580.0,0.0,13624.0,3067.0,126217.0,235667.0,ITC,ITC2,1.255010e+05
2,2022-01-02T17:00:00,3,Lombardia,2052,216,2268,289825,292093,7196,10425,...,25183247,6868839.0,15.0,993570.0,271489.0,13094013.0,12089234.0,ITC,ITC4,1.010397e+07
3,2022-01-02T17:00:00,5,Veneto,1120,188,1308,105491,106799,2908,3816,...,19758438,2472640.0,16.0,592987.0,70822.0,8333719.0,11424719.0,ITH,ITH3,4.907704e+06
4,2022-01-02T17:00:00,6,Friuli Venezia Giulia,298,28,326,14440,14766,918,1188,...,4556503,969305.0,4.0,138151.0,21454.0,2661326.0,1895177.0,ITH,ITH4,1.211357e+06
5,2022-01-02T17:00:00,7,Liguria,510,47,557,13302,13859,427,1265,...,3573911,1052169.0,3.0,152757.0,0.0,2006357.0,1567554.0,ITC,ITC3,1.543127e+06
6,2022-01-02T17:00:00,8,Emilia-Romagna,1451,130,1581,95471,97052,8186,9090,...,11738029,2407107.0,10.0,547665.0,10598.0,7329428.0,4408601.0,ITH,ITH5,4.467118e+06
7,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,9227083,3689948.0,9.0,370906.0,32054.0,5489260.0,3737823.0,ITI,ITI1,3.722729e+06
8,2022-01-02T17:00:00,10,Umbria,151,8,159,23204,23363,539,788,...,2926140,538529.0,1.0,84524.0,8944.0,1355436.0,1570704.0,ITI,ITI2,8.802850e+05
9,2022-01-02T17:00:00,11,Marche,227,50,277,7832,8109,348,699,...,2030264,1194317.0,4.0,147782.0,0.0,1654957.0,375307.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-02T17:00:00,1,Piemonte,1294,109,1403,78688,80091,1551,2619,...,7248722.0,ITC,ITC1,4.341375e+06,Piemonte,1242,5824,108,628,99
1,2022-01-02T17:00:00,2,Valle d'Aosta,36,3,39,2554,2593,114,117,...,235667.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,32,99,3,33,0
2,2022-01-02T17:00:00,3,Lombardia,2052,216,2268,289825,292093,7196,10425,...,12089234.0,ITC,ITC4,1.010397e+07,Lombardia,1945,10457,222,1530,0
3,2022-01-02T17:00:00,5,Veneto,1120,188,1308,105491,106799,2908,3816,...,11424719.0,ITH,ITH3,4.907704e+06,Veneto,1134,6000,184,1000,0
4,2022-01-02T17:00:00,6,Friuli Venezia Giulia,298,28,326,14440,14766,918,1188,...,1895177.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,275,1277,28,175,0
5,2022-01-02T17:00:00,7,Liguria,510,47,557,13302,13859,427,1265,...,1567554.0,ITC,ITC3,1.543127e+06,Liguria,494,1764,48,219,12
6,2022-01-02T17:00:00,8,Emilia-Romagna,1451,130,1581,95471,97052,8186,9090,...,4408601.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1359,9001,124,889,0
7,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,3737823.0,ITI,ITI1,3.722729e+06,Toscana,653,5033,81,570,26
8,2022-01-02T17:00:00,10,Umbria,151,8,159,23204,23363,539,788,...,1570704.0,ITI,ITI2,8.802850e+05,Umbria,133,662,8,86,41
9,2022-01-02T17:00:00,11,Marche,227,50,277,7832,8109,348,699,...,375307.0,ITI,ITI3,1.518400e+06,Marche,223,995,47,252,51


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-02T17:00:00,3,Lombardia,2052,216,2268,289825,292093,7196,10425,...,14.12,7095,-26845,44172,7550.0,45,3184,23.60,0.10318,12.52
1,2022-01-02T17:00:00,8,Emilia-Romagna,1451,130,1581,95471,97052,8186,9090,...,14.62,8088,-3165,17490,5324.0,8,894,51.97,0.20349,12.50
2,2022-01-02T17:00:00,12,Lazio,1173,152,1325,89076,90401,6920,7993,...,16.12,6902,-4352,36248,3441.0,15,1058,22.05,0.13627,8.96
3,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,14.91,5875,-8627,22513,12114.0,6,440,28.28,0.17103,10.82
4,2022-01-02T17:00:00,15,Campania,718,55,773,93605,94378,3989,5192,...,9.43,3957,-8696,29677,22246.0,3,1200,17.50,0.08974,10.41
5,2022-01-02T17:00:00,19,Sicilia,811,107,918,50378,51296,3531,3964,...,12.53,3483,-1800,22832,12015.0,13,420,17.36,0.07978,7.70
6,2022-01-02T17:00:00,5,Veneto,1120,188,1308,105491,106799,2908,3816,...,18.80,2918,-10454,24964,3595.0,13,895,15.29,0.07776,13.53
7,2022-01-02T17:00:00,16,Puglia,283,32,315,30363,30678,3311,3451,...,6.49,3305,451,16982,5702.0,0,140,20.32,0.08610,7.87
8,2022-01-02T17:00:00,1,Piemonte,1294,109,1403,78688,80091,1551,2619,...,17.36,1498,-4831,16452,4065.0,6,1062,15.92,0.06033,11.63
9,2022-01-02T17:00:00,7,Liguria,510,47,557,13302,13859,427,1265,...,21.46,412,-1167,6229,1233.0,4,834,20.31,0.08198,9.90


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,22,68,90,24.44
1,Liguria,47,172,219,21.46
2,Marche,50,202,252,19.84
3,Veneto,188,812,1000,18.80
4,Piemonte,109,519,628,17.36
5,P.A. Bolzano,17,83,100,17.00
6,Lazio,152,791,943,16.12
7,Friuli Venezia Giulia,28,147,175,16.00
8,Toscana,85,485,570,14.91
9,Calabria,28,161,189,14.81


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))